In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
warnings.filterwarnings('ignore')



In [ ]:
# --- CONFIG ---
base_dir = '../data/'
cleaned_dir = '../data/data_cleaned/'
os.makedirs(cleaned_dir, exist_ok=True)

# Chemins
bls_2023_path = os.path.join(base_dir, 'BLS/all_data_M_2023.xlsx')
bls_2024_path = os.path.join(base_dir, 'BLS/all_data_M_2024.xlsx')
irs_path = os.path.join(base_dir, 'IRS/22zpallagi.csv')

zillow_paths = {
    'metro_days': os.path.join(base_dir, 'Zillow_datasets/Metro_days_on_market_mean_doz_pending_uc_sfrcondo_sm_month.csv'),
    'metro_for_sale': os.path.join(base_dir, 'Zillow_datasets/Metro_for_sale_listings_invt_fs_uc_sfrcondo_sm_month.csv'),
    'metro_heat': os.path.join(base_dir, 'Zillow_datasets/Metro_market_heat_index_uc_sfrcondo_month.csv'),
    'metro_income_buy': os.path.join(base_dir, 'Zillow_datasets/Metro_new_homeowner_income_needed_downpayment_0.20_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'),
    'metro_income_rent': os.path.join(base_dir, 'Zillow_datasets/Metro_new_renter_income_needed_uc_sfrcondomfr_sm_sa_month.csv'),
    'metro_sales': os.path.join(base_dir, 'Zillow_datasets/Metro_sales_count_now_uc_sfrcondo_month.csv'),
    'zip_forecast': os.path.join(base_dir, 'Zillow_datasets/Zip_home_values_forecasts_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'),
    'zip_zhvi': os.path.join(base_dir, 'Zillow_datasets/Zip_home_values_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv'),
    'zip_zori': os.path.join(base_dir, 'Zillow_datasets/Zip_rentals_zori_uc_sfrcondomfr_sm_month.csv')
}

# --- Fonction melt ---
def melt_zillow_df(df, value_name='Value'):
    id_cols = [col for col in df.columns if not col.startswith(('19', '20'))]
    df_melted = pd.melt(df, id_vars=id_cols, var_name='Date', value_name=value_name)
    df_melted['Date'] = pd.to_datetime(df_melted['Date'], errors='coerce')
    df_melted = df_melted.dropna(subset=['Date', value_name])
    return df_melted

print("Début du nettoyage...\n")

# --- 1. BLS ---
print("1. Chargement BLS...")
try:
    bls_df = pd.read_excel(bls_2024_path)
    print("   → BLS 2024 chargé")
except Exception as e:
    print(f"   → Erreur BLS 2024 : {e}, fallback sur 2023")
    bls_df = pd.read_excel(bls_2023_path)

bls_cols = ['AREA_TITLE', 'OCC_TITLE', 'TOT_EMP', 'A_MEDIAN', 'A_PCT10', 'A_PCT25', 'A_PCT75', 'A_PCT90', 'LOC_QUOTIENT']
bls_df = bls_df[bls_cols].dropna(subset=['A_MEDIAN'])
bls_df['A_MEDIAN'] = pd.to_numeric(bls_df['A_MEDIAN'], errors='coerce')
bls_df['Metro'] = bls_df['AREA_TITLE'].str.replace(' MSA', '', regex=False).str.strip()

bls_metro_agg = bls_df.groupby('Metro')['A_MEDIAN'].median().reset_index().rename(columns={'A_MEDIAN': 'Median_Wage_Metro'})

bls_df.to_csv(os.path.join(cleaned_dir, 'bls_clean.csv'), index=False)
bls_metro_agg.to_csv(os.path.join(cleaned_dir, 'bls_metro_agg.csv'), index=False)
print("   → bls_clean.csv & bls_metro_agg.csv sauvegardés\n")

# --- 2. IRS ---
print("2. Chargement IRS...")
irs_df = pd.read_csv(irs_path)
irs_df = irs_df[irs_df['zipcode'] != 0]
irs_agg = irs_df.groupby('zipcode').agg(
    Total_Returns=('N1', 'sum'),
    Total_AGI=('A00100', 'sum')
).reset_index()
irs_agg['Avg_AGI'] = irs_agg['Total_AGI'] / irs_agg['Total_Returns']
irs_agg['ZIP'] = irs_agg['zipcode'].astype(str).str.zfill(5)
irs_agg = irs_agg[['ZIP', 'Avg_AGI', 'Total_Returns']]
irs_agg.to_csv(os.path.join(cleaned_dir, 'irs_clean.csv'), index=False)
print("   → irs_clean.csv sauvegardé\n")

# --- 3. Zillow ZIP ---
print("3. Chargement Zillow ZIP...")
zip_zhvi = melt_zillow_df(pd.read_csv(zillow_paths['zip_zhvi']), 'ZHVI')
zip_zori = melt_zillow_df(pd.read_csv(zillow_paths['zip_zori']), 'ZORI')
zip_forecast = melt_zillow_df(pd.read_csv(zillow_paths['zip_forecast']), 'ZHVF_Growth')

mapping_df = pd.read_csv(zillow_paths['zip_zhvi'])[['RegionName', 'Metro', 'StateName']].drop_duplicates()
mapping_df['ZIP'] = mapping_df['RegionName'].astype(str).str.zfill(5)
mapping_df = mapping_df[['ZIP', 'Metro', 'StateName']]
mapping_df.to_csv(os.path.join(cleaned_dir, 'zip_metro_mapping.csv'), index=False)

housing_zip = zip_zhvi.merge(zip_zori, on=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'Metro', 'Date'], how='outer')
housing_zip = housing_zip.merge(zip_forecast, on=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName', 'Metro', 'Date'], how='outer')
housing_zip['ZIP'] = housing_zip['RegionName'].astype(str).str.zfill(5)
housing_zip.to_csv(os.path.join(cleaned_dir, 'housing_zip_clean.csv'), index=False)
print("   → housing_zip_clean.csv & zip_metro_mapping.csv sauvegardés\n")

# --- 4. Zillow Metro ---
print("4. Chargement Zillow Metro...")
metro_dfs = {}
metro_keys = {k: v for k, v in zillow_paths.items() if 'metro' in k.lower()}

# Ordre garanti : income_buy en premier pour garantir la colonne Income_Needed_Buy
ordered_keys = [
    'metro_income_buy',   # → Income_Needed_Buy
    'metro_income_rent',  # → Income_Needed_Rent
    'metro_days',         # → DAYS
    'metro_for_sale',     # → FOR_SALE
    'metro_heat',         # → HEAT
    'metro_sales'         # → SALES
]
ordered_keys = [k for k in ordered_keys if k in metro_keys]  # Garde seulement les existants

for key in ordered_keys:
    path = metro_keys[key]
    df = pd.read_csv(path)
    
    # --- DÉTECTION PRÉCISE DU TYPE DE FICHIER ---
    filename = os.path.basename(path).lower()
    
    if 'homeowner_income_needed' in filename:
        value_name = 'Income_Needed_Buy'
    elif 'renter_income_needed' in filename:
        value_name = 'Income_Needed_Rent'
    else:
        # Pour les autres : days, for_sale, heat, sales
        part = key.split('_')[1]  # "days", "for_sale", etc.
        value_name = part.upper().replace('_', '')  # DAYS, FORSALE, HEAT, SALES
    
    # --- Melt + Nettoyage ---
    df_melted = melt_zillow_df(df, value_name)
    df_clean = df_melted[['RegionName', 'Date', value_name]].copy()
    
    # Stocker
    metro_dfs[key] = df_clean
    print(f"   → {key} → {value_name}")

# --- Merge progressif (sans duplication de colonnes) ---
base_key = ordered_keys[0]
housing_metro = metro_dfs[base_key].copy()

for key in ordered_keys[1:]:
    df = metro_dfs[key]
    value_col = df.columns[-1]  # ex: 'Income_Needed_Rent', 'DAYS', etc.
    housing_metro = housing_metro.merge(
        df[['RegionName', 'Date', value_col]],
        on=['RegionName', 'Date'],
        how='outer'
    )

# --- Ajouter la colonne Metro (une seule fois, à la fin) ---
metro_name_df = pd.read_csv(zillow_paths[base_key])[['RegionName']].drop_duplicates()
metro_name_df['Metro'] = metro_name_df['RegionName'].str.strip()
housing_metro = housing_metro.merge(metro_name_df, on='RegionName', how='left')

# --- Sauvegarde ---
housing_metro.to_csv(os.path.join(cleaned_dir, 'housing_metro_clean.csv'), index=False)
print("   → housing_metro_clean.csv sauvegardé avec succès !\n")


# --- 5. Affordability ---
print("5. Calculs d'affordability...")

# ZIP
afford_zip = housing_zip.merge(mapping_df, on='ZIP', how='left').merge(irs_agg, on='ZIP', how='left')
afford_zip['Monthly_Payment_Buy'] = (afford_zip['ZHVI'] * 0.8 * 0.07 / 12) + (afford_zip['ZHVI'] * 0.01 / 12)
afford_zip['Income_Needed_Buy'] = afford_zip['Monthly_Payment_Buy'] * 12 / 0.3
afford_zip['Income_Needed_Rent'] = (afford_zip['ZORI'] * 12) / 0.3
afford_zip['Affordability_Ratio'] = (afford_zip['Monthly_Payment_Buy'] / (afford_zip['Avg_AGI'] / 12)).replace([np.inf, -np.inf], np.nan) * 100
afford_zip['Years_To_Save_Downpayment'] = (afford_zip['ZHVI'] * 0.2) / (afford_zip['Avg_AGI'] * 0.1).replace([np.inf, -np.inf], np.nan)
afford_zip.to_csv(os.path.join(cleaned_dir, 'affordability_zip.csv'), index=False)

# Metro
afford_metro = housing_metro.merge(bls_metro_agg, on='Metro', how='left')

# Vérifier si Income_Needed_Buy existe
if 'Income_Needed_Buy' in afford_metro.columns:
    afford_metro['Afford_Ratio_vs_Wage'] = afford_metro['Income_Needed_Buy'] / afford_metro['Median_Wage_Metro']
else:
    print("   → 'Income_Needed_Buy' manquant dans housing_metro. Ratio non calculé.")
    afford_metro['Afford_Ratio_vs_Wage'] = np.nan

afford_metro.to_csv(os.path.join(cleaned_dir, 'affordability_metro.csv'), index=False)

print(f"\nNettoyage terminé ! Fichiers dans : {os.path.abspath(cleaned_dir)}")
print("Fichiers générés :")
for f in sorted(os.listdir(cleaned_dir)):
    print(f"  - {f}")

Début du nettoyage...

1. Chargement BLS...
   → BLS 2024 chargé
   → bls_clean.csv & bls_metro_agg.csv sauvegardés

2. Chargement IRS...
   → irs_clean.csv sauvegardé

3. Chargement Zillow ZIP...
   → housing_zip_clean.csv & zip_metro_mapping.csv sauvegardés

4. Chargement Zillow Metro...
   → metro_income_buy → Income_Needed_Buy
   → metro_income_rent → Income_Needed_Rent
   → metro_days → DAYS
   → metro_for_sale → FOR
   → metro_heat → HEAT
   → metro_sales → SALES
   → housing_metro_clean.csv sauvegardé avec succès !

5. Calculs d'affordability...

Nettoyage terminé ! Fichiers dans : /Users/berline/Documents/M2_Miashs/OpenData/USA_Real_Estate-Dashboard/data/data_cleaned
Fichiers générés :
  - affordability_metro.csv
  - affordability_zip.csv
  - bls_clean.csv
  - bls_metro_agg.csv
  - housing_metro_clean.csv
  - housing_zip_clean.csv
  - irs_clean.csv
  - zip_metro_mapping.csv
